In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from theano import shared
import pymc3 as pm
import pickle

In [2]:
#import data of layer ratio from Cambay
filename = '/home/julien/OneDrive/Doctorat/spatialVariability/Snowpits_Cambay/ratio_WS_DH_snowpit2015.csv'
df_cb15 = pd.read_csv(filename)
filename = '/home/julien/OneDrive/Doctorat/spatialVariability/Snowpits_Cambay/ratio_WS_DH_snowpit2016.csv'
df_cb16 = pd.read_csv(filename)
filename = '/home/julien/OneDrive/Doctorat/spatialVariability/Snowpits_Cambay/ratio_WS_DH_snowpit2017.csv'
df_cb17 = pd.read_csv(filename)
filename = '/home/julien/OneDrive/Doctorat/spatialVariability/Snowpits_Cambay/ratio_WS_DH_snowpit2018_NDVI_Sx.csv'
df_cb18 = pd.read_csv(filename)
filename = '/home/julien/OneDrive/Doctorat/spatialVariability/Snowpits_Cambay/ratio_WS_DH_snowpit2019_NDVI_Sx.csv'
df_cb19 = pd.read_csv(filename)

# TVC data
filename = '/home/julien/OneDrive/Doctorat/spatialVariability/Ratio_TVC/ratio_layer_TVC_March2018_strat_update_NDVI_TPI.csv'
df_tvc18 = pd.read_csv(filename)
filename = '/home/julien/OneDrive/Doctorat/spatialVariability/Ratio_TVC/ratio_layer_TVC_March2019_strat_update_NDVI_TPI.csv'
df_tvc19 = pd.read_csv(filename)

X = df_cb15['thickness'].append(df_cb16['thickness']).append(df_cb17['thickness']).append(df_cb18['thickness']).append(df_cb19['thickness']).append(df_tvc18['total_depth (cm)']).append(df_tvc19['total_depth (cm)'])/100
Y_DH = df_cb15['% DH'].append(df_cb16['% DH']).append(df_cb17['% DH']).append(df_cb18['% DH']).append(df_cb19['% DH']).append(df_tvc18['DH']).append(df_tvc19['DH'])
Y_WS = df_cb15['% WS'].append(df_cb16['% WS']).append(df_cb17['% WS']).append(df_cb18['% WS']).append(df_cb19['% WS']).append(df_tvc18['WS']).append(df_tvc19['WS'])

#removing NaN
df = pd.DataFrame({'x' : X, 'y_DH' : Y_DH, 'y_WS' : Y_WS})
df = df.drop(df[np.isnan(df.y_DH) == True ].index)
df.y_DH[df.y_DH == 0] =0.01
df.y_DH[df.y_DH == 1] =0.99

In [3]:
X_cb = df_cb15['thickness'].append(df_cb16['thickness']).append(df_cb17['thickness']).append(df_cb18['thickness']).append(df_cb19['thickness'])/100
X_tvc = df_tvc18['total_depth (cm)'].append(df_tvc19['total_depth (cm)'])/100
Y_DH_cb = df_cb15['% DH'].append(df_cb16['% DH']).append(df_cb17['% DH']).append(df_cb18['% DH']).append(df_cb19['% DH'])
Y_DH_tvc = df_tvc18['DH'].append(df_tvc19['DH'])

def clean_df(X, Y_DH):
    #removing NaN
    df = pd.DataFrame({'x' : X, 'y_DH' : Y_DH})
    df = df.drop(df[np.isnan(df.y_DH) == True ].index)
    df.y_DH[df.y_DH == 0] =0.01
    df.y_DH[df.y_DH == 1] =0.99
    return df
df_cb = clean_df(X_cb, Y_DH_cb)
df_tvc = clean_df(X_tvc, Y_DH_tvc)

In [4]:
# Defining the invLogit Mean Function
import theano.tensor as tt

class InvLogit(pm.gp.mean.Mean):
    """
    InvLogit function for Gaussian process. Customn Mean function
    """

    def __init__(self, a, x0, c, d):
        pm.gp.mean.Mean.__init__(self)
        self.a = a
        self.x0 = x0
        self.c = c
        self.d = d

    def __call__(self, X):
        return (self.d * pm.math.invlogit(self.a*(X - self.x0)) + self.c).reshape((-1,))

"""
begin GP Model
"""
#train model
with pm.Model() as gp_model_1:
    
    #mean_func = pm.gp.mean.Linear(coeffs=np.array([-0.2]), intercept=np.array([0.45]))
    #mean_func = pm.gp.mean.Constant(0.3)

    #custom Mean function
    mean_func = InvLogit(a = np.array([-5]), x0 = np.array([0.5]), c = np.array([0.2]), d = np.array([0.35]))
    
    def logistic(x, a, x0, c, d):
    # a is the slope, x0 is the location
        return d * pm.math.invlogit(a*(x - x0)) + c

    #params for invlogit for Cov func
    a = -5.0
    x0 = 0.6
    c = 0.25
    d = 1.5
    sigma = pm.HalfNormal('sigma', sd =0.1)
    cov_base = pm.gp.cov.WhiteNoise(0.15)
    cov_func = pm.gp.cov.ScaledCov(1, scaling_func=logistic, args=(a,x0,c,d), cov_func=cov_base)

    # Specify the GP
    gp = pm.gp.Latent(mean_func = mean_func, cov_func=cov_func)
    # make gp prior
    f = gp.prior("f", X=df.x.values.reshape(-1,1))
    p = pm.Deterministic("p", pm.math.invlogit(f))
    y = pm.Beta("y", mu = p, sigma = sigma, observed = df.y_DH.values)
    #mp = pm.find_MAP()
    gp_trace1 = pm.sample(1000, tune=1000, chains=2)
    
#set up parameter log-normal on new X with shared variables
zeta = np.sqrt(np.log(1 + 0.4**2))
lam = np.log(0.4) - (zeta**2)/2
n_sub = 500
#set up conditional and shared variable for the prediction
with gp_model_1:
    x_depth = stats.lognorm.rvs(scale = np.exp(lam),s = zeta, loc = 0, size = n_sub).reshape(-1,1)
    x_tensor = shared(value = x_depth)
    f_pred = gp.conditional(f'y_pred', x_tensor, shape = n_sub)

    

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [f_rotated_, sigma]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 27 seconds.
/home/julien/anaconda3/envs/bayes/lib/python3.7/site-packages/pymc3/gp/cov.py:93: UserWarning: Only 1 column(s) out of Subtensor{int64}.0 are being used to compute the covariance function. If this is not intended, increase 'input_dim' parameter to the number of columns to use. Ignore otherwise.
  " the number of columns to use. Ignore otherwise.", UserWarning)


In [5]:
from smrt import make_soil, sensor_list, make_model, make_snowpack, make_interface
from smrt.inputs.make_medium import make_generic_stack
from smrt.core.globalconstants import DENSITY_OF_ICE
from smrt.atmosphere.simple_isotropic_atmosphere import SimpleIsotropicAtmosphere

def ssa_to_l(ssa, density):
    #converting SSA to correlation length
    f = density/DENSITY_OF_ICE
    l_d = 4 * (1-f)/(DENSITY_OF_ICE*ssa)
    return l_d


def simu_mean(mu, temp_surface, temp_base):
    n_sub = 500
    #mu =0.34
    CV = list(np.linspace(0.1, 1, 40))
    ratio_dh = 0.46
    ratio_ws = 0.54
    density_DH = 266
    density_WS = 335

    # l_DH = kappa_37 * ssa_to_l(10, density_DH)
    # l_WS = kappa_37 * ssa_to_l(20, density_WS)
    l_DH = 1.39 * ssa_to_l(11, density_DH)
    l_WS = 1.39 * ssa_to_l(20, density_WS)

    #Tb for mean depth
    #37
    sp = make_snowpack(thickness= [mu* ratio_dh, mu*ratio_ws],
                   microstructure_model = 'exponential',
                   density = [density_WS, density_DH],
                   corr_length = [l_WS, l_DH],                 
                   temperature = [temp_surface, temp_base])
    substrate = make_soil('soil_wegmuller', complex(3.34,0.005), temperature = temp_base, roughness_rms = 0.017)
    medium = sp + substrate
    radiometer = sensor_list.passive(37e9, 55)
    model = make_model("iba", 'dort')
    res = model.run(radiometer, medium)
    print(f' 37 Mean depth TbV : {res.TbV()}, TbH : {res.TbH()}')
    
    return res.TbV(), res.TbH()


# #19
# sp = make_snowpack(thickness= [mu* ratio_dh, mu*ratio_ws],
#                microstructure_model = 'exponential',
#                density = [density_WS, density_DH],
#                corr_length = [l_WS, l_DH],                 
#                temperature = [260, 257])
# substrate = make_soil('soil_wegmuller', complex(3.34,0.005), temperature = 257, roughness_rms = 0.017)
# medium = sp + substrate
# radiometer = sensor_list.passive(19e9, 55)
# model = make_model("iba", 'dort')
# res = model.run(radiometer, medium)
# print(f' 19 Mean depth TbV : {res.TbV()}, TbH : {res.TbH()}')

In [6]:
def simu_GP(mu, temp_surface, temp_base):
#Tb fo GP process and log Normal 
    n_sub = 500
    Tb_cv_37 = []
    
    CV = list(np.linspace(0.1, 1, 40))
    ratio_dh = 0.46
    ratio_ws = 0.54
    density_DH = 266
    density_WS = 335
    l_DH = 1.39 * ssa_to_l(11, density_DH)
    l_WS = 1.39 * ssa_to_l(20, density_WS)
    
    for cv in CV:
        #fro error estimate, run 10 times
        #error_h, error_v = [], []
        #for n in range(0,10):   
        zeta = np.sqrt(np.log(1 + cv**2))
        lam = np.log(mu) - (zeta**2)/2
        swe_pred = []
        with gp_model_1:
            x_depth = stats.lognorm.rvs(scale = np.exp(lam),s = zeta, loc = 0, size = n_sub).reshape(-1,1)
            x_tensor.set_value(x_depth)
            y_samples = pm.sample_posterior_predictive(gp_trace1, vars=[f_pred], samples = 3, progressbar = False)

        DH = y_samples[f'y_pred'][1,:]
        DH = np.where(DH >= 0, DH, 0.01)
        DH = np.where(DH <= 1.0, DH, 0.99)

        radiometer = sensor_list.passive(37e9, 55)
        model = make_model("iba", 'dort')
        tbv, tbh = [], []
        for idx, d in np.ndenumerate(x_depth):
            thick = [d * DH[idx[0]], d * (1 - DH[idx[0]])]
            sp = make_snowpack(thickness= thick,
                           microstructure_model = 'exponential',
                           density = [density_WS, density_DH],
                           corr_length = [l_WS, l_DH],                 
                           temperature = [temp_surface, temp_base])
            substrate = make_soil('soil_wegmuller', complex(3.34,0.005), temperature = temp_base, roughness_rms = 0.017)
            medium = sp + substrate
            res = model.run(radiometer, medium)
            tbv.append(res.TbV())
            tbh.append(res.TbH())

        #error_h.append(np.mean(tbh))
        #error_v.append(np.mean(tbv))

        mean_tbv, mean_tbh = np.mean(tbv), np.mean(tbh)
        #std_tbv, std_tbh = np.std(error_v), np.std(error_h)
        print(f'CV : {cv}, tbv : {mean_tbv} , tbh : {mean_tbh}')
        #print(f'CV : {cv}, tbv : {mean_tbv}, tbh : {mean_tbh}')
        Tb_cv_37.append({'cv' : cv, 'mean_tbv' : mean_tbv, 'mean_tbh' : mean_tbh, 
                         'tbv' : tbv, 'tbh' : tbh})
        
    return Tb_cv_37


In [ ]:
def simu_GP(mu, temp_surface, temp_base):
#Tb fo GP process and log Normal 
    n_sub = 500
    Tb_cv_37 = []
    
    CV = list(np.linspace(0.1, 1, 40))
    ratio_dh = 0.46
    ratio_ws = 0.54
    density_DH = 266
    density_WS = 335
    l_DH = 1.39 * ssa_to_l(11, density_DH)
    l_WS = 1.39 * ssa_to_l(20, density_WS)
    
    for cv in CV:
        #fro error estimate, run 10 times
        #error_h, error_v = [], []
        #for n in range(0,10):   
        zeta = np.sqrt(np.log(1 + cv**2))
        lam = np.log(mu) - (zeta**2)/2
        swe_pred = []
        with gp_model_1:
            x_depth = stats.lognorm.rvs(scale = np.exp(lam),s = zeta, loc = 0, size = n_sub).reshape(-1,1)
            x_tensor.set_value(x_depth)
            y_samples = pm.sample_posterior_predictive(gp_trace1, vars=[f_pred], samples = 3, progressbar = False)

        DH = y_samples[f'y_pred'][1,:]
        DH = np.where(DH >= 0, DH, 0.01)
        DH = np.where(DH <= 1.0, DH, 0.99)

        radiometer = sensor_list.passive(37e9, 55)
        model = make_model("iba", 'dort')
        tbv, tbh = [], []
        for idx, d in np.ndenumerate(x_depth):
            thick = [d * DH[idx[0]], d * (1 - DH[idx[0]])]
            sp = make_snowpack(thickness= thick,
                           microstructure_model = 'exponential',
                           density = [density_WS, density_DH],
                           corr_length = [l_WS, l_DH],                 
                           temperature = [temp_surface, temp_base])
            substrate = make_soil('soil_wegmuller', complex(3.34,0.005), temperature = temp_base, roughness_rms = 0.017)
            medium = sp + substrate
            res = model.run(radiometer, medium)
            tbv.append(res.TbV())
            tbh.append(res.TbH())

        #error_h.append(np.mean(tbh))
        #error_v.append(np.mean(tbv))

        mean_tbv, mean_tbh = np.mean(tbv), np.mean(tbh)
        #std_tbv, std_tbh = np.std(error_v), np.std(error_h)
        print(f'CV : {cv}, tbv : {mean_tbv} , tbh : {mean_tbh}')
        #print(f'CV : {cv}, tbv : {mean_tbv}, tbh : {mean_tbh}')
        Tb_cv_37.append({'cv' : cv, 'mean_tbv' : mean_tbv, 'mean_tbh' : mean_tbh, 
                         'tbv' : tbv, 'tbh' : tbh})
        
    return Tb_cv_37


In [7]:
# mean depth = 0.42, temp 260, 257
TBV_cb_19, TBH_cb_19 = simu_mean(0.42, 261, 257)
# mean depth = 0.34, temp 260, 257
TBV_cb_18, TBH_cb_18 = simu_mean(0.34, 261, 257)
# mean depth = 0.42, temp 260, 257
TBV_cb_17, TBH_cb_17= simu_mean(0.42, 261, 257)
# mean depth = 0.28, temp 257, 257
TBV_cb_16, TBH_cb_16 = simu_mean(0.28, 255, 255)
# mean depth = 0.42, temp 256, 257
TBV_cb_15, TBH_cb_15 = simu_mean(0.32, 255, 253)

 37 Mean depth TbV : 186.15513203454074, TbH : 167.1910251328983
 37 Mean depth TbV : 190.39378129864025, TbH : 171.284260932607
 37 Mean depth TbV : 186.15513203454074, TbH : 167.1910251328983
 37 Mean depth TbV : 192.1336320252846, TbH : 173.20109132124063
 37 Mean depth TbV : 186.8463315727031, TbH : 168.22822803832068


In [8]:
Tb_cv_cb = simu_GP(0.42, 261, 257)
file_tb_cv = open('results_tb_cv_sd42_CB.obj', 'wb')
pickle.dump(Tb_cv_cb, file_tb_cv)

/home/julien/anaconda3/envs/bayes/lib/python3.7/site-packages/pymc3/sampling.py:1708: UserWarning: samples parameter is smaller than nchains times ndraws, some draws and/or chains may not be represented in the returned posterior predictive sample
  "samples parameter is smaller than nchains times ndraws, some draws "


CV : 0.1, tbv : 181.5200156348741 , tbh : 163.07247213568468


/home/julien/anaconda3/envs/bayes/lib/python3.7/site-packages/pymc3/sampling.py:1708: UserWarning: samples parameter is smaller than nchains times ndraws, some draws and/or chains may not be represented in the returned posterior predictive sample
  "samples parameter is smaller than nchains times ndraws, some draws "


CV : 0.12307692307692308, tbv : 181.93153990811533 , tbh : 163.46258254577435
CV : 0.14615384615384616, tbv : 183.39947544682605 , tbh : 164.84418469043885
CV : 0.16923076923076924, tbv : 182.2804177016387 , tbh : 163.80066496061002
CV : 0.19230769230769232, tbv : 182.5314159881966 , tbh : 164.04502184305875
CV : 0.2153846153846154, tbv : 182.97891268489417 , tbh : 164.4683191686226
CV : 0.23846153846153847, tbv : 182.28091442225954 , tbh : 163.84620211712217
CV : 0.26153846153846155, tbv : 182.55560183407192 , tbh : 164.05042973491305
CV : 0.2846153846153846, tbv : 184.51387678422097 , tbh : 165.9182253052909
CV : 0.3076923076923077, tbv : 184.6043202665844 , tbh : 166.00391381763097
CV : 0.3307692307692308, tbv : 183.7766124361031 , tbh : 165.21037057002937
CV : 0.3538461538461538, tbv : 185.0353867518322 , tbh : 166.38635462670004
CV : 0.3769230769230769, tbv : 185.14260540405107 , tbh : 166.50949733651944
CV : 0.4, tbv : 184.15986752740568 , tbh : 165.6154135899114
CV : 0.423076923

In [9]:
Tb_cv_cb = simu_GP(0.32, 261, 257)
file_tb_cv = open('results_tb_cv_sd32_CB.obj', 'wb')
pickle.dump(Tb_cv_cb, file_tb_cv)

CV : 0.1, tbv : 191.8814264655169 , tbh : 172.93161029546494
CV : 0.12307692307692308, tbv : 191.2512053629423 , tbh : 172.36363099924372
CV : 0.14615384615384616, tbv : 191.90052822927782 , tbh : 172.9973659233483
CV : 0.16923076923076924, tbv : 191.53861634225643 , tbh : 172.6155686086613
CV : 0.19230769230769232, tbv : 191.44406428209518 , tbh : 172.5495762353927
CV : 0.2153846153846154, tbv : 192.11528534589445 , tbh : 173.20320215909945
CV : 0.23846153846153847, tbv : 193.49465320490785 , tbh : 174.48182952510018
CV : 0.26153846153846155, tbv : 193.18018082031014 , tbh : 174.17367722005318
CV : 0.2846153846153846, tbv : 193.54222398073844 , tbh : 174.54200153455145
CV : 0.3076923076923077, tbv : 193.07870309630513 , tbh : 174.0695214025743
CV : 0.3307692307692308, tbv : 192.03711765525443 , tbh : 173.14603237030803
CV : 0.3538461538461538, tbv : 193.32118883121214 , tbh : 174.37357866229013
CV : 0.3769230769230769, tbv : 195.67827690765327 , tbh : 176.62089165786128
CV : 0.4, tbv 

## TVC

In [8]:
#Tb for mean depth for tvc
# mean depth = 0.42, temp 260, 263
TBV_tvc_19, TBH_tvc_19 = simu_mean(0.42, 260, 263)
# mean depth = 0.39, temp 260, 263
TBV_tvc_18, TBH_tvc_18 = simu_mean(0.39, 260, 263)

 37 Mean depth TbV : 189.27815650368944, TbH : 170.0762860205117
 37 Mean depth TbV : 190.69459040855952, TbH : 171.44436258332786


In [11]:
Tb_cv_tvc = simu_GP(0.41, 260, 263)
file_tb_cv_37 = open('results_tb_cv_sd41_TVC.obj', 'wb')
pickle.dump(Tb_cv_tvc, file_tb_cv_37)

CV : 0.1, tbv : 185.79002146697434 , tbh : 167.00774383628956
CV : 0.12307692307692308, tbv : 186.4793187262852 , tbh : 167.6347425574364
CV : 0.14615384615384616, tbv : 187.03377256175114 , tbh : 168.17336837898893
CV : 0.16923076923076924, tbv : 186.6445660890874 , tbh : 167.83119125802827
CV : 0.19230769230769232, tbv : 186.38537354052346 , tbh : 167.59409287238452
CV : 0.2153846153846154, tbv : 187.18684378566908 , tbh : 168.33949738765722
CV : 0.23846153846153847, tbv : 188.06861875293689 , tbh : 169.16935164460838
CV : 0.26153846153846155, tbv : 187.26236159912324 , tbh : 168.400869262003
CV : 0.2846153846153846, tbv : 186.86222596942696 , tbh : 168.0494023379072
CV : 0.3076923076923077, tbv : 188.2835730293038 , tbh : 169.38770489511447
CV : 0.3307692307692308, tbv : 188.16414819582988 , tbh : 169.22912850022811
CV : 0.3538461538461538, tbv : 187.7421749773113 , tbh : 168.86196711195865
CV : 0.3769230769230769, tbv : 188.67487697092363 , tbh : 169.81179332792527
CV : 0.4, tbv : 

### Put all simu mean in dict

In [10]:
dict_tbh = {'cb_19' : TBH_cb_19, 'cb_18' : TBH_cb_18, 'cb_17' : TBH_cb_17, 'cb_16' : TBH_cb_16,
            'cb_15' : TBH_cb_15, 'tvc_19' : TBH_tvc_19, 'tvc_18' : TBH_tvc_18}

dict_tbv = {'cb_19' : TBV_cb_19, 'cb_18' : TBV_cb_18, 'cb_17' : TBV_cb_17, 'cb_16' : TBV_cb_16,
            'cb_15' : TBV_cb_15, 'tvc_19' : TBV_tvc_19, 'tvc_18' : TBV_tvc_18}

fileh = open("Tbh_simu_mean.obj","wb")
pickle.dump(dict_tbh,fileh)

fileV = open("Tbv_simu_mean.obj","wb")
pickle.dump(dict_tbv,fileV)

### 